<a href="https://colab.research.google.com/github/kimtaekyu1204/AI_Image/blob/main/2_(DF)_%EC%84%9C%EC%9A%B8%EC%8B%9C_%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0_%EC%8B%A4%EC%8B%9C%EA%B0%84_%EB%8C%80%EC%97%AC%EC%A0%95%EB%B3%B4(JSON_%3E_DF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 서울 열린데이터 광장
서울 열린데이터 광장(Seoul Open Data Plaza)은 서울시에서 운영하는 공공데이터 개방 플랫폼입니다. 시민, 연구자, 기업 등이 서울시에서 생성한 다양한 공공데이터를 자유롭게 활용할 수 있도록 제공하고 있습니다. 이를 통해 데이터 기반의 창의적인 아이디어와 혁신을 촉진하며, 시민들의 정보 접근성을 높이고 공공서비스를 개선하는 데 기여하고 있습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

api_key_path = '/content/drive/MyDrive/AI/AI 컴퓨터비전/데이터 분석/서울 열린데이터 광장/2. 서울시 공공자전거 실시간 대여정보/Seoul_opendata_api_keys.txt'

def load_keys(file_path):
    keys = {}
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            if '=' in line:
                key, value = line.split('=', 1)
                keys[key.strip()] = value.strip()
    return keys

Keys = load_keys(api_key_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# API KEY -> 숨기기
Key = Keys['Seoul_opendata_api_keys']

In [ ]:
import requests
import folium
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

base_url = f'http://openapi.seoul.go.kr:8088/{Key}/json/bikeList/1/5/'
response = requests.get(base_url)
# print(request)
json_data = response.json()
json_data

{'rentBikeStatus': {'list_total_count': 5,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'rackTotCnt': '15',
    'stationName': '102. 망원역 1번출구 앞',
    'parkingBikeTotCnt': '11',
    'shared': '73',
    'stationLatitude': '37.55564880',
    'stationLongitude': '126.91062927',
    'stationId': 'ST-4'},
   {'rackTotCnt': '14',
    'stationName': '103. 망원역 2번출구 앞',
    'parkingBikeTotCnt': '23',
    'shared': '164',
    'stationLatitude': '37.55495071',
    'stationLongitude': '126.91083527',
    'stationId': 'ST-5'},
   {'rackTotCnt': '13',
    'stationName': '104. 합정역 1번출구 앞',
    'parkingBikeTotCnt': '3',
    'shared': '23',
    'stationLatitude': '37.55073929',
    'stationLongitude': '126.91508484',
    'stationId': 'ST-6'},
   {'rackTotCnt': '5',
    'stationName': '105. 합정역 5번출구 앞',
    'parkingBikeTotCnt': '1',
    'shared': '20',
    'stationLatitude': '37.55000687',
    'stationLongitude': '126.91482544',
    'stationId': 'ST-7'},
   {'rackTotCnt': '12',


In [ ]:
json_data.get('rentBikeStatus',{}).get("RESULT",{})

{'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'}

In [ ]:
json_data.get("rentBikeStatus", {}).get("RESULT", {}).get("CODE", "") # get(키 존재시 나옴 , 키 X시 나옴 )

'INFO-000'

In [ ]:
json_data["rentBikeStatus"]["RESULT"]["CODE"] # 위랑 같은 결과가 나오기는 함 -> 위로 접근시 접근 불가시 처리 가능 -> 안정적 작성 가능

'INFO-000'

In [ ]:
def fetch_bike_data():
    base_url = f'http://openapi.seoul.go.kr:8088/{Key}/json/bikeList/'
    start = 1
    end = 1000
    step = 1000
    data_frames = []

    while True:
        url = f"{base_url}{start}/{end}/"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Status Code: {response.status_code}")
            break

        json_data = response.json()

        try:
            rent_bike_status = json_data["rentBikeStatus"]
            result_code = rent_bike_status["RESULT"]["CODE"]
        except KeyError:
            print("JSON 오류")
            break

        if result_code == "INFO-200":
            print("데이터 없음")
            break
        elif result_code == "INFO-000":
            print(f"시작: {start} 끝: {end}.")
            try:
                bike_data = rent_bike_status["row"]
                if bike_data:
                    df = pd.DataFrame(bike_data)
                    data_frames.append(df)
            except KeyError:
                print("데이터를 찾을 수 없음")
        else:
            print(f"result code: {result_code}")
            break

        start += step
        end += step

    if data_frames:
        final_df = pd.concat(data_frames, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame()

In [ ]:
bike_data_df = fetch_bike_data()

시작: 1 끝: 1000.
시작: 1001 끝: 2000.
시작: 2001 끝: 3000.
JSON 오류


데이터 3000개 이하임

In [ ]:
bike_data_df

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,15,102. 망원역 1번출구 앞,11,73,37.55564880,126.91062927,ST-4
1,14,103. 망원역 2번출구 앞,23,164,37.55495071,126.91083527,ST-5
2,13,104. 합정역 1번출구 앞,3,23,37.55073929,126.91508484,ST-6
3,5,105. 합정역 5번출구 앞,1,20,37.55000687,126.91482544,ST-7
4,12,106. 합정역 7번출구 앞,2,17,37.54864502,126.91282654,ST-8
...,...,...,...,...,...,...,...
2727,10,6178. 마스터밸류에이스 지식산업센터,12,120,37.55875015,126.85991669,ST-3337
2728,9,6180.양천향교역 3번출구,2,22,37.56978226,126.84175110,ST-3365
2729,10,6182.오스템임플란트비전타워,25,250,37.57143402,126.82884979,ST-3370
2730,10,6183.K스퀘어 마곡,7,70,37.56269455,126.82707977,ST-3371


In [ ]:
bike_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2732 entries, 0 to 2731
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   rackTotCnt         2732 non-null   object
 1   stationName        2732 non-null   object
 2   parkingBikeTotCnt  2732 non-null   object
 3   shared             2732 non-null   object
 4   stationLatitude    2732 non-null   object
 5   stationLongitude   2732 non-null   object
 6   stationId          2732 non-null   object
dtypes: object(7)
memory usage: 149.5+ KB


In [ ]:
'''
rackTotCnt	거치대개수
parkingBikeTotCnt	자전거주차총건수
shared	거치율
stationLatitude	위도
stationLongitude	경도
stationId	대여소ID
stationName	대여소이름
'''
bike_data_df.columns

Index(['rackTotCnt', 'stationName', 'parkingBikeTotCnt', 'shared',
       'stationLatitude', 'stationLongitude', 'stationId'],
      dtype='object')

In [ ]:
bike_data_df.shape

(2732, 7)

In [ ]:
bike_data_df['stationLongitude'] = bike_data_df['stationLongitude'].astype(float)
bike_data_df['stationLatitude'] = bike_data_df['stationLatitude'].astype(float)

In [ ]:
bike_map = folium.Map(location=[bike_data_df['stationLatitude'].mean(),
                                bike_data_df['stationLongitude'].mean()],
                                zoom_start=12)

for index, data in bike_data_df.iterrows():
    popup_str = '{} 자전거주차총건수:{}대'.format(
        data['stationName'], data['parkingBikeTotCnt']
    )
    popup = folium.Popup(popup_str, max_width=600)
    folium.Marker(location=[data['stationLatitude'], data['stationLongitude']],
                  popup=popup).add_to(bike_map)

bike_map